# 1章はじめに

## 1.4 必要なライブラリとツール

#### 1.4.2 NumPy

In [15]:
import numpy as np

x = np.array([[1, 2, 3], [4, 5, 6]])
print("x:\n{}".format(x)) 

x:
[[1 2 3]
 [4 5 6]]


#### 1.4.3 SciPy

In [16]:
from scipy import sparse
# 2次元単位行列の作成
eye = np.eye(4)
print(eye)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [17]:
# Numpy行列をScipyのCSR形式の疎行列に変換
sparse_matrix = sparse.csr_matrix(eye)
print(sparse_matrix)

  (0, 0)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (3, 3)	1.0


In [18]:
# 上のものと同じ疎行列をCOO形式で作成
data = np.ones(4)
print(data)
row_indices = np.arange(4)
col_indices = np.arange(4)
eye_coo = sparse.coo_matrix((data, (row_indices, col_indices)))
print(eye_coo)

[1. 1. 1. 1.]
  (0, 0)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (3, 3)	1.0


#### 参考：Python, SciPy（scipy.sparse）で疎行列を生成・変換



https://note.nkmk.me/python-scipy-sparse-matrix-csr-csc-coo-lil/

##### SciPy（scipy.sparse）の疎行列クラスの種類


データから疎行列を生成 → COO
<br>
算術演算や行列積計算 → CSR
<br>
要素の追加・更新 → LIL
<br>
行の取得 → CSRかLIL
<br>
列の取得 → CSC
<br>
要素・部分行列の取得 → LIL

In [19]:
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix, lil_matrix

l = [
    [0, 10, 20],
    [30, 0, 40],
    [0, 0, 50]]

data = [10, 20, 30, 40, 50]
row = [0, 0, 1, 1, 2]
col = [1, 2, 0, 2, 2]

In [20]:
# CSR: scipy.sparse.csr_matrix
#Compressed Sparse Rowの略。圧縮行格納方式。最もスタンダードな方式
csr = csr_matrix(l)
print(csr)
print('値:', csr.data)
print('列:',csr.indices)
print('行のリストが圧縮:',csr.indptr)
#i行のデータがdata, indicesのindptr[i]:indptr[i+1]に格納されていることを意味している。
#例の場合、0行目のデータが0:2、1行目のデータが2:4、2行目のデータが4:5。

  (0, 1)	10
  (0, 2)	20
  (1, 0)	30
  (1, 2)	40
  (2, 2)	50
値: [10 20 30 40 50]
列: [1 2 0 2 2]
行のリストが圧縮: [0 2 4 5]


In [21]:
# CSC: scipy.sparse.csc_matrix
# Compressed Sparse Columnの略。圧縮列格納方式。
csc = csc_matrix(l)
print(csc)
print('値:', csc.data)
print('行:',csc.indices)
print('列のリストが圧縮:',csc.indptr)
# CSCはCSRの列バージョン。
# CSRは行単位でデータが左から右、上から下の順番で格納されていたが、CSCは列単位なので上から下、左から右の順番となる。
# indptrの圧縮の考え方はCSRと同じ。

  (1, 0)	30
  (0, 1)	10
  (0, 2)	20
  (1, 2)	40
  (2, 2)	50
値: [30 10 20 40 50]
行: [1 0 0 1 2]
列のリストが圧縮: [0 1 2 5]


In [22]:
# COO: scipy.sparse.coo_matrix COOはCoordinateの略。
coo = coo_matrix(l)
print(coo)
print('値:', csc.data)
print('行:',coo.row)
print('列:',coo.col)
# 値、および、行・列のインデックスのリストがそのまま格納されている。
# オブジェクトの生成は高速だが算術演算などはサポートされていない。
# CSRやCSCへの変換も高速なので、COOを生成後、CSRに変換し演算を行うのが基本的な使い方。

  (0, 1)	10
  (0, 2)	20
  (1, 0)	30
  (1, 2)	40
  (2, 2)	50
値: [30 10 20 40 50]
行: [0 0 1 1 2]
列: [1 2 0 2 2]


##### SciPyで疎行列を生成・変換(リスト、numpy.ndarrayと相互変換)

In [49]:
# リストと相互変換
l = [[0, 10, 20],
     [30, 0, 40],
     [0, 0, 0]]
csr = csr_matrix(l)
print('リストから作成\n', csr)
print(type(csr))
print(csr.shape)

リストから作成
   (0, 1)	10
  (0, 2)	20
  (1, 0)	30
  (1, 2)	40
<class 'scipy.sparse.csr.csr_matrix'>
(3, 3)


In [53]:
# numpy.ndarrayからscipy.sparseのクラスに変換
a = np.array(l)
print('numpy配列\n', a)
print(type(a))

csr = csr_matrix(a)
print('numpyから作成\n', csr)


numpy配列
 [[ 0 10 20]
 [30  0 40]
 [ 0  0  0]]
<class 'numpy.ndarray'>
numpyから作成
   (0, 1)	10
  (0, 2)	20
  (1, 0)	30
  (1, 2)	40


In [56]:
# 1次元のリストやnumpy.ndarrayは1行の行列とみなされる。3次元以上だとエラー。
print('1次元のリスト\n', csr_matrix([0, 1, 2]))
print(csr_matrix([0, 1, 2]).shape)

1次元のリスト
   (0, 1)	1
  (0, 2)	2
(1, 3)


In [59]:
# scipy.sparseのクラスからnumpy.ndarrayに変換
# toarray()メソッド
print(csr.toarray())
print(type(csr.toarray()))

[[ 0 10 20]
 [30  0 40]
 [ 0  0  0]]
<class 'scipy.sparse.csr.csr_matrix'>


In [61]:
# scipy.sparseのクラスからリストに変換
# tolist()
print(csr.toarray().tolist())
print(type(csr.toarray().tolist()))


[[0, 10, 20], [30, 0, 40], [0, 0, 0]]
<class 'list'>


値、行、列のリストから生成
<br>
値、行、列のリストがそれぞれdata, row, column

In [63]:
# scipy.sparseからnumpy.matrixに変換
# todense()
print(csr.todense())
print(type(csr.todense()))

[[ 0 10 20]
 [30  0 40]
 [ 0  0  0]]
<class 'numpy.matrix'>


##### SciPyで疎行列を生成・変換(値、行、列のリストから生成)

In [79]:
# 値、行、列のリストがそれぞれdata, row, column
data = [10, 20, 30, 40]
row = [0, 0, 1, 1]
col = [1, 2, 0, 2]

print('scipy.sparse\n', csr_matrix((data, (row, col))))
print(type(csr_matrix((data, (row, col)))))
print('numpy.ndarray\n',csr_matrix((data, (row,col))).toarray())
print(type(csr_matrix((data, (row,col))).toarray()))
print('numpy.ndarrayで形状指定\n', csr_matrix((data, (row, col)), shape=(3, 3)).toarray())



scipy.sparse
   (0, 1)	10
  (0, 2)	20
  (1, 0)	30
  (1, 2)	40
<class 'scipy.sparse.csr.csr_matrix'>
numpy.ndarray
 [[ 0 10 20]
 [30  0 40]]
<class 'numpy.ndarray'>
numpy.ndarrayで形状指定
 [[ 0 10 20]
 [30  0 40]
 [ 0  0  0]]
